### Calculate deaths from PM2.5

In [4]:
import geopandas as gpd
import xarray as xr
from cartopy import crs as ccrs
import seaborn as sns; sns.set_theme()
import os
import fiona
import country_converter as coco
import dask
import dask.array as da
import netCDF4 as nc
import regionmask
from matplotlib import cm
import numpy as np
from matplotlib import pyplot as plt
import country_converter as coco
import pyogrio
#pyogrio.set_gdal_config_options({"SHAPE_RESTORE_SHX": "YES"})
import pandas as pd
from cartopy.util import add_cyclic_point
import nc_time_axis
import glob
import cdo
import pandas as pd
import cartopy.feature as cfeature


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Gridded Population of the World, Version 4 (GPWv4): National Identifier Grid, Revision 11
# can be downloaded from: https://www.earthdata.nasa.gov/data/catalog/sedac-ciesin-sedac-gpwv4-natiden-r11-4.11#:~:text=Description,use%20in%20aggregating%20population%20data.
# cited as: Center For International Earth Science Information Network-CIESIN-Columbia University. (2018). Gridded Population of the World, Version 4 (GPWv4): National Identifier Grid, Revision 11 (Version 4.11) [Data set]. Palisades, NY: NASA Socioeconomic Data and Applications Center (SEDAC). https://doi.org/10.7927/H4TD9VDP
shapefile_path = "./gpw-v4-national-identifier-grid-rev11_15_min_asc/gpw_v4_national_identifier_grid_rev11_15_min.shp"

# Read the shapefile
gdf = gpd.read_file(shapefile_path)
#gdf_list = pd.read_csv(f'./Mortality_data/country_borders/gpw_new.csv')

In [8]:
# read in mortality data that has been verified

#mortality = xr.open_mfdataset(f'/glade/derecho/scratch/cindywang625/BMR/*.nc', parallel=True, combine='nested', concat_dim='time')['mortality']
mortality_ncds_female = xr.open_mfdataset(f'./bmr/NCDS/female/*.nc', parallel=True, combine='nested', concat_dim='time')['mortality']
mortality_ncds_male = xr.open_mfdataset(f'./bmr/NCDS/male/*.nc', parallel=True, combine='nested', concat_dim='time')['mortality']

mortality_lris_female = xr.open_mfdataset(f'./bmr/LRIS/female/*.nc', parallel=True, combine='nested', concat_dim='time')['mortality']
mortality_lris_male = xr.open_mfdataset(f'./bmr/LRIS/male/*.nc', parallel=True, combine='nested', concat_dim='time')['mortality']

#mortality_cardio = xr.open_mfdataset(f'/glade/derecho/scratch/cindywang625/BMR/for_o3/cardio/*.nc', parallel=True, combine='nested', concat_dim='time')['mortality']
#mortality_resp = xr.open_mfdataset(f'/glade/derecho/scratch/cindywang625/BMR/for_o3/resp/*.nc', parallel=True, combine='nested', concat_dim='time')['mortality']

#mortality_og = pd.read_csv(f'./Mortality_data/IFs/mortality_all_new.csv')

#mortality per thousand
demographic_female = xr.open_mfdataset(f'./demographic/countries/female_2020_demo_frac.nc')['demo_pop']
demographic_male = xr.open_mfdataset(f'./demographic/countries/male_2020_demo_frac.nc')['demo_pop']

#demographic = demographic.sum("age")

#used to make a mask
population = xr.open_dataset(f'./population/SSP2/Total/NetCDF/ssp2_2020.nc')


In [9]:
#get country mask
lon = population['lon']
lat = population['lat']
print(lat.shape)
mask = regionmask.mask_3D_geopandas(gdf,lon,lat)

(1117,)


In [10]:
age_theta = {
    '25to29': 0.1585,
    '30to34': 0.1577,
    '35to39': 0.1570,
    '40to44': 0.1558,
    '45to49': 0.1532,
    '50to54': 0.1499,
    '55to59': 0.1462,
    '60to64': 0.1421,
    '65to69': 0.1374,
    '70to74': 0.1319,
    '75to79': 0.1253,
    '80to84': 0.1141
}

### Calculating health impacts associated with air pollution (PM2.5)

Calculate mortalities attributed to PM2.5 from noncommunicable diseases and lower respiratory infections (combined) based on epidemiological cohort studies of long-term exposure to PM2.5

\begin{equation}
M = BMR \times P \times AF
\end{equation}

- M = mortality due to PM2.5
- BMR = baseline mortality rate
- P = exposed population
- AF = attributable factor

((mortality_ncds_female* demographic_female) + (mortality_ncds_male* demographic_male)
(mortality_lris_female* demographic_female) + (mortality_lris_male* demographic_male))*AF

In [11]:
mortality = ((mortality_ncds_female* (demographic_female.values/1e3))+(mortality_ncds_male* (demographic_male.values/1e3)))+(
    (mortality_lris_female* (demographic_female.values/1e3))+(mortality_lris_male* (demographic_male.values/1e3)))



In [51]:
#AF = xr.open_mfdataset('/glade/derecho/scratch/cindywang625/RR/003.nc')['RR_base']-xr.open_mfdataset('/glade/derecho/scratch/cindywang625/RR/003.nc')['RR_em']
#AF = xr.open_mfdataset('/glade/derecho/scratch/cindywang625/RR/pm245_so4/avg_RR.nc')['RR_diff']
ensemble = 'death'#'010'

#AF = - xr.open_mfdataset(f'/Volumes/bkup_srm/SRM/RR/SSP245/{ensemble}.nc')['RR_base'][:,5:] + xr.open_mfdataset(f'/Volumes/bkup_srm/SRM/RR/SSP245/{ensemble}.nc')['RR_base'][:,:10].mean("time")
#AF = - xr.open_mfdataset(f'/Volumes/bkup_srm/SRM/RR/{ensemble}.nc')['RR_em'] + xr.open_mfdataset(f'/Volumes/bkup_srm/SRM/RR/SSP245/{ensemble}.nc')['RR_base'][:,:10].mean("time")


#AF_0 = 1-xr.open_mfdataset(f'/glade/derecho/scratch/cindywang625/RR/SSP245_2030_2039/{ensemble}.nc')['RR_base']
AF = 1- xr.open_mfdataset(f'/Volumes/bkup_srm/SRM/RR/ssp245/{ensemble}.nc')['RR_base'][:,5:]#+xr.open_mfdataset('/glade/derecho/scratch/cindywang625/RR/010.nc')['RR_base'][:,0]


In [52]:
deaths= (AF.transpose('time','age','lat', 'lon'))*mortality[5:].values

In [53]:
deaths_val = deaths.where(deaths != 0, np.nan)


In [54]:
#ensemble = 'deaths_ensemble'
#deaths_val.to_netcdf(f'/glade/derecho/scratch/cindywang625/deaths_ozone/arise_2035/by_grid/{ensemble}.nc')
#deaths_val.to_netcdf(f'/glade/derecho/scratch/cindywang625/deaths_ozone/arise_ssp/by_grid/{ensemble}.nc')
deaths_val.to_netcdf(f'/Volumes/bkup_srm/SRM/deaths/ssp245/by_grid/{ensemble}.nc')


In [50]:
num_ensembles = 10
for i in range(1, num_ensembles + 1):
    ensemble = f'{i:03d}' 
    print(f'Loading ensemble member {ensemble}')
    AF = 1- xr.open_mfdataset(f'/Volumes/bkup_srm/SRM/RR/{ensemble}.nc')['RR_em']
    deaths= (AF.transpose('time','age','lat', 'lon'))*mortality[5:].values
    deaths_val = deaths.where(deaths != 0, np.nan)

    deaths_val.to_netcdf(f'/Volumes/bkup_srm/SRM/deaths/arise/by_grid/{ensemble}.nc')

    

Loading ensemble member 001
Loading ensemble member 002
Loading ensemble member 003
Loading ensemble member 004
Loading ensemble member 005
Loading ensemble member 006
Loading ensemble member 007
Loading ensemble member 008
Loading ensemble member 009
Loading ensemble member 010
